# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [1]:
import altair as alt
import pandas as pd

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [2]:
def legislators_df():
    # Load the data
    legislators_df = pd.read_csv('data/legislators.csv')

    # Extract the state from jurisdiction column
    legislators_df['state'] = legislators_df['jurisdiction'].apply(
        lambda x: x.split('state:')[1].split('/')[0] if 'state:' in x 
        else x.split('district:')[1].split('/')[0] if 'district:' in x 
        else x.split('territory:')[1].split('/')[0] if 'territory:' in x 
        else None
    )

    return legislators_df
    
# render dataframe
legislators_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,dc
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,dc
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,dc
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,dc
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,dc
...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,pr
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,pr
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,pr
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,pr


### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [3]:
def group_data(df, group_cols):
    '''Function groups the data by specified columns'''
    return df.groupby(group_cols).size().reset_index(name='count')

def states_by_gender_initial(df):
    # Group data by state and gender
    gender_counts = group_data(df, ['state', 'gender'])

    # Create the stacked bar chart
    chart = alt.Chart(gender_counts).mark_bar().encode(
        x=alt.X('state:N', title='State', sort=alt.SortField('state', order='ascending')),
        y=alt.Y('sum(count):Q', title='Number of Legislators'),
        color=alt.Color('gender:N', title='Gender'),
        tooltip=['state', 'gender', 'count']
    ).properties(
        width=1000,
        height=500
    ).configure_axis(
        labelAngle=0
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )

    return chart.display()

# render the chart
states_by_gender_initial(legislators_df())

alt.Chart(...)

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [4]:
def calculate_totals(grouped_df, primary_group_col):
    '''This function calculates the total counts for each grouping column'''
    return grouped_df.groupby(primary_group_col)['count'].sum().reset_index(name='total')

def calculate_percentages(grouped_df, totals_df, primary_group_col):
    '''This function calculates percentages needed for stacked bars'''
    merged_df = grouped_df.merge(totals_df, on=primary_group_col)
    merged_df['percentage'] = merged_df['count'] / merged_df['total']

    return merged_df

def reference_line():
    '''Create 50% line for reference'''
    line = alt.Chart(pd.DataFrame({'y': [.50]})).mark_rule(
        color='black'
    ).encode(y='y:Q')

    return line

def combine_chart_elements(ref_line, bars):
    '''Combine stacked bars and 50% line'''
    chart = (bars + ref_line).properties(
        width=1000,
        height=500
    ).configure_axis(
        labelAngle=0
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )
    
    return chart.display()

def states_by_gender_improved(df):
    # Group data by state and gender
    gender_counts = group_data(df, ['state', 'gender'])
    
    # Calculate total legislators by state
    state_totals = calculate_totals(gender_counts, 'state')
    
    # Merge dataframes and calculate percentages
    gender_percentages = calculate_percentages(gender_counts, state_totals, 'state')
    
    # Create base chart
    base = alt.Chart(gender_percentages).encode(
        x=alt.X('state:N', 
                title='State', 
                sort=alt.SortField(field='state', order='ascending')),
        tooltip=['state', 'gender']
    )
    
    # Create stacked bar chart
    bars = base.mark_bar().encode(
        y=alt.Y('percentage:Q', 
                title='Percentage of Legislators',
                axis=alt.Axis(format='%')),
        color=alt.Color('gender:N', 
                        title='Gender',
                        scale=alt.Scale(domain=['Female', 'Male'],
                                       range=['#8624f5', '#1fc3aa']))
    )
    
    # Create 50% line
    line = reference_line()
    
    # Combine elements & return final chart
    return combine_chart_elements(line, bars)

# render the chart
states_by_gender_improved(legislators_df())

alt.LayerChart(...)

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [5]:
def party_control_raw(df):
    # Group data by state and party
    party_counts = group_data(df, ['state', 'party'])
    
    # Calculate total legislators per state
    state_totals = calculate_totals(party_counts, 'state')
    
    # Merge dataframes and calculate percentages
    party_percentages = calculate_percentages(party_counts, state_totals, 'state')
    
    # Create base chart
    base = alt.Chart(party_percentages).encode(
        x=alt.X('state:N', 
                title='State', 
                sort=alt.SortField(field='state', order='ascending')),
        tooltip=['state', 'party']
    )
    
    # Create stacked bar chart
    bars = base.mark_bar().encode(
        y=alt.Y('percentage:Q', 
                title='Percentage of Legislators',
                axis=alt.Axis(format='%')),
        color=alt.Color('party:N', 
                        title='Party')
    )
    
    # Create 50% line
    line = reference_line()
    
    # Combine elements & return final chart
    return combine_chart_elements(line, bars)

# render the chart
party_control_raw(legislators_df())

alt.LayerChart(...)

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [6]:
def clean_party_df():
    # Load original legislator data
    df = legislators_df()

    # Add party_code column to dataframe
    df['party_code'] = df['party'].apply(lambda x: 'D' if 'Democratic' in x 
                                         else 'R' if 'Republican' in x 
                                         else 'O')

    # Drop outliers: 'DC', 'NE', and 'PR'
    df_cleaned = df[~df['state'].isin(['dc', 'ne', 'pr'])]

    return df_cleaned

clean_party_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state,party_code
13,Kevin McCabe,Kevin J.,McCabe,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,30,lower,ak,R
14,C.J. McCormick,Conrad J.,McCormick,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,38,lower,ak,D
15,Bill Wielechowski,Bill P.,Wielechowski,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,K,upper,ak,D
16,Justin Ruffridge,Justin,Ruffridge,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,7,lower,ak,R
17,Will Stapp,Will,Stapp,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,32,lower,ak,R
...,...,...,...,...,...,...,...,...,...,...
7353,Karlee Provenza,Karlee R.,Provenza,Female,Democratic,ocd-jurisdiction/country:us/state:wy/government,45,lower,wy,D
7354,Affie Ellis,Affie Burnside,Ellis,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,8,upper,wy,R
7355,Abby Angelos,Abby,Angelos,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,3,lower,wy,R
7356,Pepper Ottman,Pepper L.,Ottman,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,34,lower,wy,R


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

**NOTE TO GRADER:**

I wasn't able to figure out how to create two separate charts for each chamber type like in  `imgs/ex2.2.png`. I tried to add facet the chart like instructed in the `combine_elements_w_facet()` function above, but I kept getting an error that said "ValueError: Facet charts require data to be specified at the top level" and I could not figure out how to fix it even with the help of Stackoverflow threads like this one: https://stackoverflow.com/questions/67318821/bar-chart-in-altair-valueerror-faceted-charts-cannot-be-layered

In [7]:
def combine_elements_w_facet(ref_line, bars):
    '''
    Combine stacked bars and 50% line 
    I COULDN'T GET THIS FUNCTION TO WORK
    '''
    chart = (bars + ref_line).properties(
        width=1000,
        height=500
    ).configure_axis(
        labelAngle=0
    ).facet(
        column='type:N'
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )
    
    return chart.display()

def party_control_by_chamber(df):
    # Group data by state, type, and party_code
    party_counts = group_data(df, ['state', 'type', 'party_code'])
    
    # Calculate totals
    state_chamber_totals = calculate_totals(party_counts, ['state', 'type'])
    
    # Merge dataframes and calculate percentages
    party_percentages = calculate_percentages(party_counts, state_chamber_totals, ['state', 'type'])
    
    # Create bars
    bars = alt.Chart(party_percentages).mark_bar().encode(
        y=alt.Y('state:N', title='State', sort='-x'),
        x=alt.X('percentage:Q', 
                title='Percentage of Legislators',
                scale=alt.Scale(domain=[0, 1])),
        color=alt.Color('party_code:N',
                        scale=alt.Scale(domain=['D', 'O', 'R'],
                                        range=['#0015BC', '#00FF00', '#FF0000']),
                        legend=alt.Legend(title='Party')),
        tooltip=['state', 'type', 'party_code']
    )
    
    # Create 50% line
    line = reference_line()
    
    # Combine elements & return final chart
    return combine_chart_elements(line, bars)

# Display the chart
party_control_by_chamber(clean_party_df())

alt.LayerChart(...)

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [8]:
def population_combined_df():
    # Load data
    legislature_df = legislators_df()
    population_df = pd.read_csv('data/populations.csv')
    
    # Group by 'state' and 'type'
    seat_counts = legislature_df.groupby(['state', 'type']).size().unstack(fill_value=0)
    
    # Rename columns to 'upper' and 'lower'
    seat_counts.columns = seat_counts.columns.str.replace('upper', 'upper').str.replace('lower', 'lower')
    
    # Merge the dataframes
    combined_df = pd.merge(seat_counts.reset_index(), population_df, on='state', how='left')
    
    return combined_df[['state', 'upper', 'lower', 'pop_2020']]

population_combined_df().head()

,state,upper,lower,pop_2020
0,ak,20,40,733374
1,al,35,104,5024294
2,ar,35,98,3011490
3,az,30,59,7157902
4,ca,40,79,39538212


### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [9]:
def scatter_pop_size():
    combined_df = population_combined_df()

    # Create base chart
    base = alt.Chart(combined_df).encode(
        x=alt.X('pop_2020:Q', title='Population (2020)'),
        y=alt.Y('seats:Q', title='Number of Seats'),
        tooltip=['state', 'pop_2020', 'seats']
    ).properties(
        width=800,
        height=500
    )

    # Upper chamber chart
    upper_chart = base.mark_point(
        shape='triangle-up',
        size=100,
        color='purple'
    ).encode(
        y='upper:Q',
        tooltip=['state', 'upper', 'pop_2020']
    ).properties()

    # Lower chamber chart
    lower_chart = base.mark_point(
        shape='triangle-down',
        size=100,
        color='orange'
    ).encode(
        y='lower:Q',
        tooltip=['state', 'lower', 'pop_2020']
    ).properties()

    # Combine both chart layers
    combined_chart = upper_chart + lower_chart

    # Add labels
    combined_chart = combined_chart.configure_axis(
        labelFontSize=12,
        titleFontSize=14
    )

    return combined_chart

scatter_pop_size().display()

alt.LayerChart(...)

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [10]:
def scatter_pop_size_regression():
    combined_df = population_combined_df()
    original_scatter_plot = scatter_pop_size() # call previous function
    
    # Regression line for lower chamber (orange)
    lower_regression = alt.Chart(combined_df).mark_line(
        color='orange'
    ).transform_regression(
        'pop_2020', 'lower'
    ).encode(
        x='pop_2020:Q',
        y='lower:Q'
    )

    # Regression line for upper chamber (purple)
    upper_regression = alt.Chart(combined_df).mark_line(
        color='purple'
    ).transform_regression(
        'pop_2020', 'upper'
    ).encode(
        x='pop_2020:Q',
        y='upper:Q'
    )

    # Combine original scatter plot and regression layers
    combined_chart = original_scatter_plot + upper_regression + lower_regression

    # Add axis labels
    combined_chart = combined_chart.configure_axis(
        labelFontSize=12,
        titleFontSize=14
    )

    return combined_chart.display()


scatter_pop_size_regression()

alt.LayerChart(...)

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [11]:
def actions_df():
    # Load actions data
    df = pd.read_csv('data/actions_il-in-mi-wi_2021-2024.csv')
    
    # Clean up date column
    df['date'] = pd.to_datetime(df['date'].str[:10], format='%Y-%m-%d')
    
    return df

# Verify that data is properly loaded
actions_df().head()

,description,date,classification,state,session
0,"Filed with the Clerk by Rep. Marcus C. Evans, Jr.",2021-02-10,['filing'],IL,102nd
1,Filed with Secretary,2021-08-26,['filing'],IL,102nd
2,Filed with Secretary by Sen. Jacqueline Y. Col...,2021-02-25,['filing'],IL,102nd
3,Filed with Secretary,2022-01-19,['filing'],IL,102nd
4,Filed with Secretary,2022-11-29,['filing'],IL,102nd


### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [12]:
def actions_heatmap_scaled(df):
    # Group data by state and year-week
    df_grouped = group_data(df, [df['state'], df['date'].dt.strftime('%Y W%U')])

    # Create a list of dates for labels on x-axis (every 13 weeks)
    all_dates = sorted(df_grouped['date'].unique())
    tick_dates = all_dates[::13]
    
    # Create heatmap chart
    heatmap = alt.Chart(df_grouped).mark_rect().encode(
        x=alt.X('date:O', title='Date (Year-Week)', 
                axis=alt.Axis(labelAngle=0, 
                              values = tick_dates,
                              tickCount=len(tick_dates))),
        y=alt.Y('state:O', title='state'),
        color=alt.Color('count:Q', scale=alt.Scale(scheme='yellowgreenblue'), title='actions'),
        tooltip=['state', 'date', 'count']
    ).properties(
        width=1200,
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_legend(
        titleFontSize=13,
        labelFontSize=12
    )
    
    return heatmap.display()

actions_heatmap_scaled(actions_df())

alt.Chart(...)

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [13]:
df = actions_df()
df_exclude_il = df[df['state'] != 'IL']
actions_heatmap_scaled(df_exclude_il)

alt.Chart(...)

In [14]:
df = actions_df()
df_only_il = df[df['state'] == 'IL']
actions_heatmap_scaled(df_only_il)

alt.Chart(...)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [15]:
def actions_cumulative(df):
    # Group data by state and date
    df_grouped = group_data(df, ['state', 'date'])
    
    # Create line chart
    line_chart = alt.Chart(df_grouped).transform_window(
        cumulative_actions='sum(count)',
        groupby=['state'],
        sort=[{'field': 'date'}]
    ).mark_line().encode(
        x=alt.X('date:T', title='Date'),
        y=alt.Y('cumulative_actions:Q', title='Cumulative Actions'),
        color='state:N',
        tooltip=['state', 'date:T', 'cumulative_actions:Q']
    ).properties(
        width=700,
        height=400
    )
    
    return line_chart.display()

actions_cumulative(actions_df())

alt.Chart(...)